Streaming data dan data preprocessing Kelompok 5 PDB

12S18002 : Wiranda Siahaan

12S18009 : Novita Enjelia Hutapea

12S18041 : Merika H. Manurung

12S18046 : Tiara Octavia Situmorang

# Consume twitter steaming data

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
import socket
import json
import numpy as np
import pandas as pd

In [3]:
#Kindly put your credentials received from twitter developer account
consumer_key='2mTmxHwkZramjVDOewE2dybgQ'
consumer_secret='dJ42E93prTAze5BoF4wAIcxYA6gXfu9rGrHr6gguUt85xcHJSi'
access_token ='1517128047019040768-YMZCcFIf6VLbZXNkJsa79sZk70xFkM'
access_secret='Auwa9WPIktjUCrgQmzceZsdMtMA2hbnebSR7QlmYEdrlq'

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [5]:
for tweet in api.search(q='miniso', lan='en'):    
    print(tweet.text)

@sriputrimhrni @ohmybeautybank Aku beli di miniso offcial store nya kak, kyknya ada si di OS tpi gtau deh masih ada atau nggak
Hot take 
miniso Is trash .
พร้อมส่ง❕
Miniso x Sanrio
รถเข็น
ราคาอันละ 145 บาท รวมส่ง

สั่งซื้อ/สอบถามเพิ่มเติมDM
มาได้เลยนะคะ
#ตลาดนัดsanrio ต… https://t.co/vKCsA4zrhC
RT @PcxPreorder: ☁️ Miniso X Cinnamoroll ☁️
ของใหม่เข้าเยอะมาก! น่ารักจนใจเจ็บ
.
💖ค่าส่งเหมาๆ 35.- ไม่บวกเพิ่ม
🍼พรีปิดรอบทุกวันที่10/20/30…
@JSBroto @RimArema Parfum apa boneka babi itu merk miniso?
RT @OneDaySS_: นี่ชอบโฆษณาของ Miniso มากเลย แบบว่าน้องเฟยอวี่หล่อใสมากกก มีความพราวเสน่ห์วิ้งคึ ซุกซน ขี้เล่นนิดๆ อ่ยยย ใจสั่น!! #ChenFeiyu…
@JSBroto Loh ! Kamu pelanggan miniso juga ? Pantes km bisa diandalkan
@kumisaurus Tumbler starbucks. Tas adidas. Sepatu nike. Nangis w jadi mas2 scbd mahal bener:( sepatu w diadora trs… https://t.co/daUVfrTDx9
@Leucosia13 @teamisurus @GavottoLA @seiyalol Es de miniso :3
RT @cheronniza: MINISO tas rantai 🌼
https://t.co/1T5GvEerSJ https://t.co/CfnsjIWghO
RT @chero

In [6]:
tweetDf = pd.DataFrame(columns = ['User', 'User_statuses_count', 
                             'user_followers','fav_count','User_location','Tweets',])

In [7]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100000, lang='en').items():
        print(i, end='\r')
        tweetDf.loc[i, 'User'] = tweet.user.name
        tweetDf.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        tweetDf.loc[i, 'user_followers'] = tweet.user.followers_count
        tweetDf.loc[i, 'fav_count'] = tweet.favorite_count
        tweetDf.loc[i, 'User_location'] = tweet.user.location
        tweetDf.loc[i, 'Tweets'] = tweet.text
        
        
        tweetDf.to_csv('{}.csv'.format(file_name))
        i+=1
        if i == 200:
            break
        else:
            pass

In [8]:
stream(data = ['miniso'], file_name = 'miniso_2021')

In [9]:
tweetDf.head()

,User,User_statuses_count,user_followers,fav_count,User_location,Tweets
0,fireynoodles🍜,1178,273,0,Disappointment Island,Hot take \nminiso Is trash .
1,jayel ♡˖꒰ᵕ•ᵕ⑅꒱,3449,46,0,Magic shop ♡,RT @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 Xiumin...
2,all thing's about idol shopee🫡,4059,118,0,,miniso garden of mirror\n\n🖇️ https://t.co/sRa...
3,Makcee Oscar,562,254,0,"Nairobi, Kenya","RT @MinisoKE: Ah,its Monday, we meet Again😍\n\..."
4,Makcee Oscar,562,254,0,"Nairobi, Kenya",RT @MinisoKE: Miniso has ✨Everything you don't...


## Data Preprocessing

In [10]:
# libraries
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
from sklearn.impute import SimpleImputer # used for handling missing data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # used for encoding categorical data
from sklearn.model_selection import train_test_split # used for splitting training and testing data
from sklearn.preprocessing import StandardScaler # used for feature scaling

In [11]:
dataset = pd.read_csv('miniso_2021.csv') # to import the dataset into a variable
# Splitting the attributes into independent and dependent attributes
X = dataset.iloc[:, :-1].values # attributes to determine dependent variable / Class
Y = dataset.iloc[:, -1].values # dependent variable / Class

In [12]:
dataset.head()

,Unnamed: 0,User,User_statuses_count,user_followers,fav_count,User_location,Tweets
0,0,fireynoodles🍜,1178,273,0,Disappointment Island,Hot take \nminiso Is trash .
1,1,jayel ♡˖꒰ᵕ•ᵕ⑅꒱,3449,46,0,Magic shop ♡,RT @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 Xiumin...
2,2,all thing's about idol shopee🫡,4059,118,0,NaN,miniso garden of mirror\n\n🖇️ https://t.co/sRa...
3,3,Makcee Oscar,562,254,0,"Nairobi, Kenya","RT @MinisoKE: Ah,its Monday, we meet Again😍\n\..."
4,4,Makcee Oscar,562,254,0,"Nairobi, Kenya",RT @MinisoKE: Miniso has ✨Everything you don't...


In [13]:
print(dataset.shape)

(200, 7)


### missing value

In [14]:
# handling the missing data and replace missing values with nan from numpy and replace with mean of all the other values
dataset1 = dataset.dropna()
print(dataset1.shape)

(143, 7)


### remove duplicate

In [15]:
dataset2 = dataset1.drop_duplicates()
print(dataset2.shape)

(143, 7)


# Text Preprocessing



In [16]:
import re, string, unicodedata
import json
import nltk
import json
import time
import itertools  
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import tensorflow.keras.preprocessing.text as kpt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

## Case folding

In [17]:
# Change text column to lower lower case
dataset2['Tweets'] = dataset2['Tweets'].str.lower()

# Look at the change
print(dataset2)

     Unnamed: 0            User  User_statuses_count  user_followers  \
0             0   fireynoodles🍜                 1178             273   
1             1  jayel ♡˖꒰ᵕ•ᵕ⑅꒱                 3449              46   
3             3    Makcee Oscar                  562             254   
4             4    Makcee Oscar                  562             254   
9             9         Mr Khan                 4740             242   
..          ...             ...                  ...             ...   
194         194               🗯                 1502              45   
195         195             kei                  145               7   
196         196           Jay 🌽                 7137             618   
197         197       ♡͛ も *。゜🎀               117179             626   
199         199   . . . kei jay                33536             554   

     fav_count          User_location  \
0            0  Disappointment Island   
1            0          Magic shop ♡    
3           

## Remove URL

In [18]:
def clean_data(dataset2):
#replace URL of a text
    dataset2['Tweets'] = dataset2['Tweets'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')

clean_data(dataset2)
print(dataset2['Tweets']);

0                           hot take \nminiso is trash .
1      rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
3      rt @minisoke: ah,its monday, we meet again😍\n\...
4      rt @minisoke: miniso has ✨everything you don't...
9      rt @amazonmallpk: get these beautiful and hand...
                             ...                        
194    rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
195    @kkieyii lets go when i go home there sa manil...
196    rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
197    rt @sugarsanriio: hello kitty cooler bag from ...
199    rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
Name: Tweets, Length: 143, dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


## Remove Number

In [19]:
# removing numbers from strings of speciafied 
# column, here 'Name'
dataset2['Tweets']= dataset2['Tweets'].str.replace('\d+', '')
  
# display output with numbers removed from 
# required strings
print(dataset2)

     Unnamed: 0            User  User_statuses_count  user_followers  \
0             0   fireynoodles🍜                 1178             273   
1             1  jayel ♡˖꒰ᵕ•ᵕ⑅꒱                 3449              46   
3             3    Makcee Oscar                  562             254   
4             4    Makcee Oscar                  562             254   
9             9         Mr Khan                 4740             242   
..          ...             ...                  ...             ...   
194         194               🗯                 1502              45   
195         195             kei                  145               7   
196         196           Jay 🌽                 7137             618   
197         197       ♡͛ も *。゜🎀               117179             626   
199         199   . . . kei jay                33536             554   

     fav_count          User_location  \
0            0  Disappointment Island   
1            0          Magic shop ♡    
3           

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


## Remove Punctuation

In [20]:
dataset2['Tweets'] = dataset2['Tweets'].str.replace(r'[^\w\s]','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [21]:
print(dataset2)

     Unnamed: 0            User  User_statuses_count  user_followers  \
0             0   fireynoodles🍜                 1178             273   
1             1  jayel ♡˖꒰ᵕ•ᵕ⑅꒱                 3449              46   
3             3    Makcee Oscar                  562             254   
4             4    Makcee Oscar                  562             254   
9             9         Mr Khan                 4740             242   
..          ...             ...                  ...             ...   
194         194               🗯                 1502              45   
195         195             kei                  145               7   
196         196           Jay 🌽                 7137             618   
197         197       ♡͛ も *。゜🎀               117179             626   
199         199   . . . kei jay                33536             554   

     fav_count          User_location  \
0            0  Disappointment Island   
1            0          Magic shop ♡    
3           

## Stopwords Removal

In [22]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [23]:
print(dataset2['Tweets']); 

import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
 
dataset2['Tweets'] = dataset2['Tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
print(dataset2['Tweets'])

0                            hot take \nminiso is trash 
1      rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
3      rt minisoke ahits monday we meet again\n\nupda...
4      rt minisoke miniso has everything you dont thi...
9      rt amazonmallpk get these beautiful and handy ...
                             ...                        
194    rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
195    kkieyii lets go when i go home there sa manila...
196    rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
197    rt sugarsanriio hello kitty cooler bag from mi...
199    rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
Name: Tweets, Length: 143, dtype: object
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
0                                  hot take miniso trash
1      rt kyvo_gvr wts lfb help rt 𓂋 xiumin exopla

## Tokenizing

In [24]:
dataset2['Tweets'] = dataset2['Tweets'].apply(word_tokenize)

print(dataset2['Tweets'])

0                             [hot, take, miniso, trash]
1      [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
3      [rt, minisoke, ahits, monday, meet, update, ma...
4      [rt, minisoke, miniso, everything, dont, think...
9      [rt, amazonmallpk, get, beautiful, handy, cros...
                             ...                        
194    [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
195    [kkieyii, lets, go, go, home, sa, manila, even...
196    [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
197    [rt, sugarsanriio, hello, kitty, cooler, bag, ...
199    [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
Name: Tweets, Length: 143, dtype: object


In [25]:
print(dataset2)

     Unnamed: 0            User  User_statuses_count  user_followers  \
0             0   fireynoodles🍜                 1178             273   
1             1  jayel ♡˖꒰ᵕ•ᵕ⑅꒱                 3449              46   
3             3    Makcee Oscar                  562             254   
4             4    Makcee Oscar                  562             254   
9             9         Mr Khan                 4740             242   
..          ...             ...                  ...             ...   
194         194               🗯                 1502              45   
195         195             kei                  145               7   
196         196           Jay 🌽                 7137             618   
197         197       ♡͛ も *。゜🎀               117179             626   
199         199   . . . kei jay                33536             554   

     fav_count          User_location  \
0            0  Disappointment Island   
1            0          Magic shop ♡    
3           

## Hasil Akhir

In [26]:
dataset2.to_csv('/content/gdrive/My Drive/pdb/preprocessing_PDB.csv', index = False)